# Feature Engineering: bag of words

In [24]:
#import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
#from sklearn.base import TransformerMixin
#from sklearn.pipeline import Pipeline
#from sklearn.svm import LinearSVC
#from sklearn.linear_model import SGDClassifier
#from sklearn.metrics import accuracy_score
#from sklearn.model_selection import train_test_split
#from sklearn.multiclass import OneVsRestClassifier
#from sklearn.metrics import confusion_matrix
#from sklearn.metrics import classification_report
#from gensim import corpora, models
#from textblob import TextBlob
#import operator
#import mord

from src import *

## Data file

In [2]:
# Read data
db_file = './data/db_master.pickle'
df = pd.read_pickle(db_file)

In [3]:
# Take a subset for testing
df = df[:5]

In [4]:
df.shape

(5, 6)

In [5]:
# Pre-clean text
% time df = pre_clean_text(df)

CPU times: user 593 ms, sys: 10 ms, total: 603 ms
Wall time: 635 ms


src/text_preparation.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['precleantext'][i] = clean_text
src/text_preparation.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['difftext'][i] = diff_text


In [6]:
df

,urn,name,url,ofsted,rawtext,scraping,precleantext,difftext
0,100322,Avonmore Primary School,http://www.avonmore.lbhf.sch.uk/,2.0,NOLINKABOUT,-1,,
1,100324,Brackenbury Primary School,http://www.brackenbury.lbhf.sch.uk/,1.0,"[Video: A Day in the Life of Brackenbury, Prov...",1,"[Video: A Day in the Life of Brackenbury, Prov...",[]
2,100326,Miles Coverdale Primary School,http://www.milescoverdaleprimary.co.uk/,1.0,[We want every child to fulfil their potential...,1,[We want every child to fulfil their potential...,[]
3,100328,Flora Gardens Primary School,http://www.floragardens.lbhf.sch.uk/,3.0,"[Flora Gardens Primary School, Dalling Road, H...",1,"[Flora Gardens Primary School, Dalling Road, H...","[Please click here to view our prospectus, Cur..."
4,100331,Kenmont Primary School,http://www.kenmont-primary.org/,2.0,[Kenmont has three main community languages - ...,1,[Kenmont has three main community languages - ...,[World Book Day Thursday 2nd March 2017Februar...


In [7]:
# Tokenize
%time df = tokenize_text(df)

src/text_preparation.py:145: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['tokens'][i] = tokens


CPU times: user 14.2 s, sys: 2.71 s, total: 16.9 s
Wall time: 36.9 s


In [8]:
df.columns.values

array(['urn', 'name', 'url', 'ofsted', 'rawtext', 'scraping',
       'precleantext', 'difftext', 'tokens'], dtype=object)

In [9]:
df = df.drop(['scraping', 'difftext'], axis=1)

In [10]:
df = df.drop(['url', 'rawtext'], axis=1)

In [11]:
df.columns.values

array(['urn', 'name', 'ofsted', 'precleantext', 'tokens'], dtype=object)

In [16]:
df['tokens'][0]

''

In [8]:
db_file_clean = './data/db_master_clean.pickle'
df.to_pickle(db_file_clean)

## Read data file

In [49]:
db_file = './data/db_master_clean.pickle'
df = pd.read_pickle(db_file)

In [50]:
df.columns.values

array(['urn', 'name', 'ofsted', 'cleantext', 'tokens'], dtype=object)

In [17]:
df.shape

(5, 5)

In [18]:
# Merge tokens to a single string

for i in range(0,df.shape[0]):
#for i in range(0,4):
    tokens = df['tokens'][i]
    text = ''
    for j in range(0, len(tokens)):
        if (j == 0):
            text += ''.join(tokens[j])
        else:
            text += ''.join(' ' + tokens[j])
    df['tokens'][i] = str(text)

/usr/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## Bag of words

### Vectorization

In [20]:
vectorizer = CountVectorizer(ngram_range=(1, 1), stop_words=None, min_df=0.0, max_df=1.0)
X = vectorizer.fit_transform(df['tokens'])
y = df['ofsted']

In [22]:
#vectorizer.get_feature_names()

In [23]:
X.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 3, 0, ..., 1, 1, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [2, 0, 0, ..., 1, 0, 6],
       [1, 1, 1, ..., 0, 0, 1]])

In [ ]:
def create_features_bow(X, min_df=0.0, max_df=1.0):
    
    def print_feature_report(X_tfidf, min_df, max_df): 
        print ">>> feature report:"
        X_length, _ = X_tfidf[:,0].shape
        print "%d texts" % X_length
        print "required minimal word occurence of %.1f" % min_df
        print "required maximal word occurence of %.1f" % max_df
        print "global number of different words in texts:", X_tfidf.count_nonzero()
    
    ##### 1) Clean and Tokenize Text
    # write first 4 web_scraped text in raw, cleaned and tokenized form
    # to files './data/nlp/raw.txt', './data/nlp/cleaned.txt' and 
    # './data/nlp/tokenized.txt' for inspection 
    #write_example_text_to_file(X,'raw',4)
    #write_example_text_to_file(X,'cleaned',4)
    #write_example_text_to_file(X,'tokenized',4)
    #print '>>> Raw text: \n', X[0][:100]
    #print '\n>>> Cleaned text: \n', cleanText(X[0])[:100]
    #print '\n>>> Cleaned and tokenized text (first 10 tokens): \n', tokenizeText(cleanText(X[0]))[:10]

    ##### 2) Feature Vectorization of Words ("Bag of Words")
    # count_vectorizer = class that contains dictionary of vocabulary etc.
    # X_counts = (sparse) feature matrix that counts the number of appearances in X
    count_vectorizer, X_counts = initiate_count_vectorizer(X, min_df=min_df, max_df=max_df)
    #print X_counts.count_nonzero()
    #print X_counts.data[:4]

    #### 3) Tf-Idf Transformation
    tfidf_transformer, X_tfidf = initiate_tfidf_transformer(X_counts)
    #print X_tfidf.count_nonzero()
    #print X_tfidf.data[:4]
    
    print_feature_report(X_tfidf, min_df, max_df)
    
    return X_tfidf, X_counts, count_vectorizer, tfidf_transformer